In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import os

print(tf.VERSION)
print(tf.__version__)
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)

sess = tf.Session(config=config)

set_session(sess)  # set this TensorFlow session as the default session for Keras

1.10.0
1.10.0


Using TensorFlow backend.


In [2]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
 #import keras

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images

from skimage.measure import compare_psnr, compare_ssim


from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

#import tensorflow as tf

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout, UpSampling2D
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img



from keras.layers import Input, Conv2D, Activation, BatchNormalization
from keras.layers.merge import Add
from keras.layers.core import Dropout
#import tensorflow as tf

from keras.models import Model
from keras.engine import InputSpec
from keras.engine.topology import Layer
from keras.layers import Input, Conv2D, Activation, BatchNormalization
from keras.layers.merge import Add
from keras.utils import conv_utils
from keras.layers.core import Dropout

from keras.layers import Input, Activation, Add
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.core import Lambda
from keras.layers.normalization import BatchNormalization
from keras.models import Model


from keras.layers import Input
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D
from keras.layers.core import Dense, Flatten
from keras.layers.normalization import BatchNormalization
from keras.models import Model

image_shape = (448,896,1)


In [3]:

def res_block(input, filters, kernel_size=(3,3), strides=(1,1), use_dropout=False):
    """
    Instanciate a Keras Resnet Block using sequential API.
    :param input: Input tensor
    :param filters: Number of filters to use
    :param kernel_size: Shape of the kernel for the convolution
    :param strides: Shape of the strides for the convolution
    :param use_dropout: Boolean value to determine the use of dropout
    :return: Keras Model
    """
    x = ReflectionPadding2D((1,1))(input)
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               strides=strides,)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    if use_dropout:
        x = Dropout(0.5)(x)

    x = ReflectionPadding2D((1,1))(x)
    x = Conv2D(filters=filters,
                kernel_size=kernel_size,
                strides=strides,)(x)
    x = BatchNormalization()(x)

    # Two convolution layers followed by a direct connection between input and output
    merged = Add()([input, x])
    
    return merged

class ReflectionPadding2D(Layer):
    """Reflection-padding layer for 2D input (e.g. picture).
    This layer can add rows and columns or zeros
    at the top, bottom, left and right side of an image tensor.
    # Arguments
        padding: int, or tuple of 2 ints, or tuple of 2 tuples of 2 ints.
            - If int: the same symmetric padding
                is applied to width and height.
            - If tuple of 2 ints:
                interpreted as two different
                symmetric padding values for height and width:
                `(symmetric_height_pad, symmetric_width_pad)`.
            - If tuple of 2 tuples of 2 ints:
                interpreted as
                `((top_pad, bottom_pad), (left_pad, right_pad))`
        data_format: A string,
            one of `channels_last` (default) or `channels_first`.
            The ordering of the dimensions in the inputs.
            `channels_last` corresponds to inputs with shape
            `(batch, height, width, channels)` while `channels_first`
            corresponds to inputs with shape
            `(batch, channels, height, width)`.
            It defaults to the `image_data_format` value found in your
            Keras config file at `~/.keras/keras.json`.
            If you never set it, then it will be "channels_last".
    # Input shape
        4D tensor with shape:
        - If `data_format` is `"channels_last"`:
            `(batch, rows, cols, channels)`
        - If `data_format` is `"channels_first"`:
            `(batch, channels, rows, cols)`
    # Output shape
        4D tensor with shape:
        - If `data_format` is `"channels_last"`:
            `(batch, padded_rows, padded_cols, channels)`
        - If `data_format` is `"channels_first"`:
            `(batch, channels, padded_rows, padded_cols)`
    """

    def __init__(self,
                 padding=(1, 1),
                 data_format=None,
                 **kwargs):
        super(ReflectionPadding2D, self).__init__(**kwargs)
        self.data_format = conv_utils.K.normalize_data_format(data_format)
        if isinstance(padding, int):
            self.padding = ((padding, padding), (padding, padding))
        elif hasattr(padding, '__len__'):
            if len(padding) != 2:
                raise ValueError('`padding` should have two elements. '
                                 'Found: ' + str(padding))
            height_padding = conv_utils.normalize_tuple(padding[0], 2,
                                                        '1st entry of padding')
            width_padding = conv_utils.normalize_tuple(padding[1], 2,
                                                       '2nd entry of padding')
            self.padding = (height_padding, width_padding)
        else:
            raise ValueError('`padding` should be either an int, '
                             'a tuple of 2 ints '
                             '(symmetric_height_pad, symmetric_width_pad), '
                             'or a tuple of 2 tuples of 2 ints '
                             '((top_pad, bottom_pad), (left_pad, right_pad)). '
                             'Found: ' + str(padding))
        self.input_spec = InputSpec(ndim=4)

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            if input_shape[2] is not None:
                rows = input_shape[2] + self.padding[0][0] + self.padding[0][1]
            else:
                rows = None
            if input_shape[3] is not None:
                cols = input_shape[3] + self.padding[1][0] + self.padding[1][1]
            else:
                cols = None
            return (input_shape[0],
                    input_shape[1],
                    rows,
                    cols)
        elif self.data_format == 'channels_last':
            if input_shape[1] is not None:
                rows = input_shape[1] + self.padding[0][0] + self.padding[0][1]
            else:
                rows = None
            if input_shape[2] is not None:
                cols = input_shape[2] + self.padding[1][0] + self.padding[1][1]
            else:
                cols = None
            return (input_shape[0],
                    rows,
                    cols,
                    input_shape[3])

    def call(self, inputs):
        return spatial_reflection_2d_padding(inputs,
                                             padding=self.padding,
                                             data_format=self.data_format)

    def get_config(self):
        config = {'padding': self.padding,
                  'data_format': self.data_format}
        base_config = super(ReflectionPadding2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    
def spatial_reflection_2d_padding(x, padding=((1, 1), (1, 1)), data_format=None):
    """
    Pad the 2nd and 3rd dimensions of a 4D tensor.
    :param x: Input tensor
    :param padding: Shape of padding to use
    :param data_format: Tensorflow vs Theano convention ('channels_last', 'channels_first')
    :return: Tensorflow tensor
    """
    assert len(padding) == 2
    assert len(padding[0]) == 2
    assert len(padding[1]) == 2
    if data_format is None:
        data_format = image_data_format()
    if data_format not in {'channels_first', 'channels_last'}:
        raise ValueError('Unknown data_format ' + str(data_format))

    if data_format == 'channels_first':
        pattern = [[0, 0],
                   [0, 0],
                   list(padding[0]),
                   list(padding[1])]
    else:
        pattern = [[0, 0],
                   list(padding[0]), list(padding[1]),
                   [0, 0]]
    return tf.pad(x, pattern, "REFLECT")

In [4]:

#from keras.layers import ReflectionPadding2D, res_block

ngf = 64
input_nc = 1
output_nc = 1
input_shape_generator = (40, 60, input_nc)
n_blocks_gen = 16


def generator_model():
    """Build generator architecture."""
    # Current version : ResNet block
    inputs = Input(shape=image_shape)

    x = ReflectionPadding2D((3, 3))(inputs)
    x = Conv2D(filters=ngf, kernel_size=(7,7), padding='valid')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Increase filter number
    n_downsampling = 2
    for i in range(n_downsampling):
        mult = 2**i
        x = Conv2D(filters=ngf*mult*2, kernel_size=(3,3), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    # Apply 9 ResNet blocks
    mult = 2**n_downsampling
    for i in range(n_blocks_gen):
        x = res_block(x, ngf*mult, use_dropout=True)

    # Decrease filter number to 3 (RGB)
    for i in range(n_downsampling):
        mult = 2**(n_downsampling - i)
        x = Conv2DTranspose(filters=int(ngf * mult / 2), kernel_size=(3,3), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

    x = ReflectionPadding2D((3,3))(x)
    x = Conv2D(filters=output_nc, kernel_size=(7,7), padding='valid')(x)
    x = Activation('tanh')(x)

    # Add direct connection from input to output and recenter to [-1, 1]
    outputs = Add()([x, inputs])
    outputs = Lambda(lambda z: z/2)(outputs)

    model = Model(inputs=inputs, outputs=outputs, name='Generator')
    return model

In [5]:

ndf = 64
output_nc = 1
input_shape_discriminator = (40, 60, output_nc)


def discriminator_model():
    """Build discriminator architecture."""
    n_layers, use_sigmoid = 3, False
    inputs = Input(shape=input_shape_discriminator)

    x = Conv2D(filters=ndf, kernel_size=(4,4), strides=2, padding='same')(inputs)
    x = LeakyReLU(0.2)(x)

    nf_mult, nf_mult_prev = 1, 1
    for n in range(n_layers):
        nf_mult_prev, nf_mult = nf_mult, min(2**n, 8)
        x = Conv2D(filters=ndf*nf_mult, kernel_size=(4,4), strides=2, padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)

    nf_mult_prev, nf_mult = nf_mult, min(2**n_layers, 8)
    x = Conv2D(filters=ndf*nf_mult, kernel_size=(4,4), strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(filters=1, kernel_size=(4,4), strides=1, padding='same')(x)
    if use_sigmoid:
        x = Activation('sigmoid')(x)

    x = Flatten()(x)
    x = Dense(1024, activation='tanh')(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=x, name='Discriminator')
    return model

In [6]:
from keras.layers import Input
from keras.models import Model

def generator_containing_discriminator_multiple_outputs(generator, discriminator):
    inputs = Input(shape=image_shape)
    generated_images = generator(inputs)
    outputs = discriminator(generated_images)
    model = Model(inputs=inputs, outputs=[generated_images, outputs])
    return model

In [7]:
import tensorflow as tf
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.models import Model
from  keras.layers import Concatenate
#import tf.image.ssim_multiscale

image_shape = (40,60,1)

def perceptual_loss(y_true, y_pred):
    vgg = VGG16(include_top=False,   input_shape=(448,896,3) )
    y_true_c =   Concatenate()([y_true,y_true,y_true])    
    y_pred_c =   Concatenate()([y_pred,y_pred,y_pred])    

    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    return K.mean(K.square(loss_model(y_true_c) - loss_model(y_pred_c)))

def DSSIM_loss(y_true, y_pred):
    # source: https://gist.github.com/Dref360/a48feaecfdb9e0609c6a02590fd1f91b
    vgg = VGG16(include_top=False,   input_shape=(448,896,3) )
    y_true_c =   Concatenate()([y_true,y_true,y_true])    
    y_pred_c =   Concatenate()([y_pred,y_pred,y_pred])    

    loss_model = Model(inputs=vgg.input, outputs=vgg.get_layer('block3_conv3').output)
    loss_model.trainable = False
    p_loss = K.mean(K.square(loss_model(y_true_c) - loss_model(y_pred_c)))
    s_loss = -1*tf.reduce_mean(tf.image.ssim(y_true, y_pred, 2.0))
    
    loss = 0.5*(p_loss+s_loss)
    return loss
    
def wasserstein_loss(y_true, y_pred):
    #return K.mean(K.square(y_true)-K.square(y_pred))
    return K.mean(y_true*y_pred)

In [8]:
# Set some parameters
im_width = 896
im_height = 448
border = 5
path_train = 'train/'
path_test = 'test/'
from sklearn.feature_extraction import image
from patchify import patchify, unpatchify
import scipy.misc

def extract_patches(image,shape):
    print(image.shape)
    patches = patchify(image,(shape[0],shape[1],1),5)
    s = patches.shape
    return patches.reshape(s[0]*s[1]*s[2],s[3],s[4],s[5])

def extract_patches(image):
    import tensorflow as tf
 
    ksize_rows = 40
    ksize_cols = 60
    strides_rows = 5
    strides_cols = 5

    sess = tf.InteractiveSession()

 
    # The size of sliding window
    ksizes = [1, ksize_rows, ksize_cols, 1] 

    # How far the centers of 2 consecutive patches are in the image
    strides = [1, strides_rows, strides_cols, 1]

 
    rates = [1, 1, 1, 1] # sample pixel consecutively

    # padding algorithm to used
    padding='VALID' # or 'SAME'

    image = tf.expand_dims(image, 0)
    #image = image.reshape((1,448,896,1))
    #print(image.shape)
    image_patches = tf.extract_image_patches(image, ksizes, strides, rates, padding)

    # print image shape of image patches
    #print(sess.run(tf.shape(image_patches)))

    # image_patches is 4 dimension array, you can use tf.squeeze to squeeze it, e.g.
    # image_patches = tf.squeeze(image_patches)

    
    # retrieve the 1st patches
    patch1 = image_patches[0,0,0,]
    patch2 = image_patches[0,0,10,]

    # reshape
    patches = tf.reshape(image_patches, [image_patches.shape[1],image_patches.shape[2],ksize_rows, ksize_cols,1])
    patch1 = patches[0,:,:,0]
    patch2 = tf.reshape(patch2,[40,60])
    #print(patch1.shape)
    #scipy.misc.imsave('GAN/patches/i1.png',sess.run(patch2 ))
    # visualize image
    patches = sess.run(patches)
    '''
    for i in range(50):
        scipy.misc.imsave('GAN/patches/'+str(i)+'.png',patches[0,i,:,:,0])
        #scipy.misc.imsave('GAN/patches/'+str(i)+'_test.png',patches[i,:,:,0])
 
        if i==5:
            break
    '''
    sess.close()
    patches = patches.reshape(patches.shape[0]*patches.shape[1],patches.shape[2],patches.shape[3],patches.shape[4])
    print(patches.shape)
    
    return patches

def get_data(path):
    ids = next(os.walk(path + "raw"))[2]
    
    X = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)
    y = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)
 
    print('Getting and resizing images ... ')
    for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
        # Load raw
        img = load_img(path + 'raw/' + id_, color_mode = "grayscale")
        x_img = img_to_array(img)
        x_img = resize(x_img, (448,896, 1), mode='constant', preserve_range=True)
 
        
        if n==0:
            X_patches = extract_patches (x_img )
        else:
            np.append(X_patches,extract_patches (x_img ))
        
        # Load average
 
        average = img_to_array(load_img(path + 'average/' + id_, color_mode = "grayscale"))
        average = resize(average, (448, 896, 1), mode='constant', preserve_range=True)
        #average = average/255
        if n==0:
            y_patches = extract_patches (average )
        else:
            np.append(y_patches, extract_patches (average ))

        # Save images
        X[n] = x_img  / 255
 
        y[n] = average / 255
        #print(X_patches.shape)
        #if n == 2:
        #scipy.misc.imsave('GAN/i1.png',X_patches[160,:,:,0])
        #scipy.misc.imsave('GAN/i2.png',y_patches[160,:,:,0])
        #scipy.misc.imsave('GAN/i1.png',X_patches[0,:,:,0])
        #scipy.misc.imsave('GAN/i2.png',y_patches[0,:,:,0])

        #    break
    print('Done!')
 
    return X_patches , y_patches
 
    
x_train, y_train = get_data(path_train )

Getting and resizing images ... 


A Jupyter Widget

(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 60, 1)
(13776, 40, 

In [ ]:
import os
import datetime
import click
import numpy as np
import tqdm
from keras.callbacks import TensorBoard
from keras.optimizers import Adam

def save_all_weights(d, g, epoch_number, current_loss):
    now = datetime.datetime.now()
    save_dir = os.path.join('GAN/16_RES/PATCH/', '{}{}'.format(now.month, now.day))
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    g.save_weights(os.path.join(save_dir, 'generator_{}_{}.h5'.format(epoch_number, current_loss)), True)
    d.save_weights(os.path.join(save_dir, 'discriminator_{}.h5'.format(epoch_number)), True)


def train_multiple_outputs(n_images, batch_size, log_dir, epoch_num, critic_updates=5):
 
    g = generator_model()
    #g =build_res_unet()
    #g.load_weights('GAN/16_RES/410/generator_69_36.h5')

    d = discriminator_model()
    d_on_g = generator_containing_discriminator_multiple_outputs(g, d)

    d_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    d_on_g_opt = Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    d.trainable = True
    d.compile(optimizer=d_opt, loss=wasserstein_loss)
    d.trainable = False
    #loss = [perceptual_loss, wasserstein_loss]
    loss = [DSSIM_loss, wasserstein_loss]

    loss_weights = [100, 5]
    d_on_g.compile(optimizer=d_on_g_opt, loss=loss, loss_weights=loss_weights)
    d.trainable = True

    output_true_batch, output_false_batch = np.ones((batch_size, 1)), -np.ones((batch_size, 1))

    log_path = './logs'
    tensorboard_callback = TensorBoard(log_path)

    for epoch in tqdm.tqdm(range(epoch_num)):
        permutated_indexes = np.random.permutation(x_train.shape[0])

        d_losses = []
        d_on_g_losses = []
        for index in range(int(x_train.shape[0] / batch_size)):
            batch_indexes = permutated_indexes[index*batch_size:(index+1)*batch_size]
            image_blur_batch = x_train[batch_indexes]
            image_full_batch = y_train[batch_indexes]

            generated_images = g.predict(x=image_blur_batch, batch_size=batch_size)

            for _ in range(critic_updates):
                d_loss_real = d.train_on_batch(image_full_batch, output_true_batch)
                d_loss_fake = d.train_on_batch(generated_images, output_false_batch)
                d_loss = 0.5 * np.add(d_loss_fake, d_loss_real)
                d_losses.append(d_loss)

            d.trainable = False

            d_on_g_loss = d_on_g.train_on_batch(image_blur_batch, [image_full_batch, output_true_batch])
            d_on_g_losses.append(d_on_g_loss)

            d.trainable = True

        # write_log(tensorboard_callback, ['g_loss', 'd_on_g_loss'], [np.mean(d_losses), np.mean(d_on_g_losses)], epoch_num)
        print(np.mean(d_losses), np.mean(d_on_g_losses))
        with open('log.txt', 'a+') as f:
            f.write('{} - {} - {}\n'.format(epoch, np.mean(d_losses), np.mean(d_on_g_losses)))

        save_all_weights(d, g, epoch, int(np.mean(d_on_g_losses)))
    

train_multiple_outputs(n_images=551040, batch_size=32, log_dir="lg", epoch_num=25, critic_updates=5)



Using TensorFlow backend.


In [13]:
def extract_patches(image):
    import tensorflow as tf
 
    ksize_rows = 200
    ksize_cols = 300
    strides_rows = 50
    strides_cols = 50

    sess = tf.InteractiveSession()

 
    # The size of sliding window
    ksizes = [1, ksize_rows, ksize_cols, 1] 

    # How far the centers of 2 consecutive patches are in the image
    strides = [1, strides_rows, strides_cols, 1]

 
    rates = [1, 1, 1, 1] # sample pixel consecutively

    # padding algorithm to used
    padding='VALID' # or 'SAME'

    image = tf.expand_dims(image, 0)
    #image = image.reshape((1,448,896,1))
    print(image.shape)
    image_patches = tf.extract_image_patches(image, ksizes, strides, rates, padding)

    # print image shape of image patches
    print(sess.run(tf.shape(image_patches)))

    # image_patches is 4 dimension array, you can use tf.squeeze to squeeze it, e.g.
    # image_patches = tf.squeeze(image_patches)

    
    # retrieve the 1st patches
    patch1 = image_patches[0,0,0,]
    patch2 = image_patches[0,0,10,]

    # reshape
    patches = tf.reshape(image_patches, [image_patches.shape[1],image_patches.shape[2],ksize_rows, ksize_cols,1])
    patch1 = patches[0,:,:,0]
    patch2 = tf.reshape(patch2,[200,300])
    print(patch1.shape)
    scipy.misc.imsave('GAN/patches/i1.png',sess.run(patch2 ))
    # visualize image
    patches = sess.run(patches)
    
    for i in range(50):
        scipy.misc.imsave('GAN/patches/'+str(i)+'.png',patches[0,i,:,:,0])
        #scipy.misc.imsave('GAN/patches/'+str(i)+'_test.png',patches[i,:,:,0])
 
        if i==5:
            break
    sess.close()
    
    return patches

average = img_to_array(load_img( 'train/average/1.tif', color_mode = "grayscale"))
average = resize(average, (448, 896, 1), mode='constant', preserve_range=True)

extract_patches(average)

(1, 448, 896, 1)
[    1     5    12 60000]
(12, 200, 1)


/home/rupali/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:45: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
/home/rupali/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


array([[[[[ 65.02673091],
          [ 69.99785754],
          [ 70.95969687],
          ...,
          [ 72.32676479],
          [ 74.66517857],
          [ 71.64140226]],

         [[ 72.        ],
          [ 66.06040238],
          [ 72.93318519],
          ...,
          [ 78.63354492],
          [ 73.65177575],
          [ 71.98214286]],

         [[ 73.98670679],
          [ 68.11085977],
          [ 77.85603177],
          ...,
          [ 68.35330636],
          [ 71.38468092],
          [ 71.03348214]],

         ...,

         [[ 91.34242965],
          [ 85.87362982],
          [102.10016243],
          ...,
          [ 82.64606585],
          [ 82.19896464],
          [ 78.2720175 ]],

         [[ 88.34618144],
          [ 90.43265705],
          [ 90.75041354],
          ...,
          [ 84.03121512],
          [ 89.98361268],
          [ 80.29910714]],

         [[ 95.12175642],
          [ 93.6167341 ],
          [ 85.45476423],
          ...,
          [ 82.45483398],
 

In [12]:
 
for i in range(50):
    scipy.misc.imsave('GAN/patches/'+str(i)+'_train.png',x_train[i,:,:,0])
    scipy.misc.imsave('GAN/patches/'+str(i)+'_test.png',y_train[i,:,:,0])
 
    if i==50:
        break

/home/rupali/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/rupali/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  after removing the cwd from sys.path.
